In [ ]:
# Fix environment compatibility issue
!pip install protobuf==3.20.3 -q

In [1]:
import numpy as np 
import pandas as pd 
import os
import re
from math import log2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, AutoTokenizer
from transformers import AutoTokenizer
import torch.nn as nn
from transformers import RobertaModel
from torch.optim import AdamW
from transformers import get_linear_schedule_with_warmup
import time
import datetime
from sklearn.metrics import accuracy_score, f1_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
import pickle
from IPython.display import FileLink

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
# Custom
import kagglehub
# Group 1: Datasets that have only 3 colunms (Subject, Body, Label)
# Download latest version
email_path = kagglehub.dataset_download("naserabdullahalam/phishing-email-dataset")
try:
    df_enron = pd.read_csv(f'{email_path}/Enron.csv', encoding='latin-1')
    df_ling = pd.read_csv(f'{email_path}/Ling.csv', encoding='latin-1')
except FileNotFoundError as e:
    print(f"Error loading file for Merge 1: {e}")

# Group 2: Rich Metadata Datasets that have 7 colunms (Sender, Date, Subject, Body, Label, etc.)
try:
    df_ceas = pd.read_csv(f'{email_path}/CEAS_08.csv', encoding='latin-1')
    df_nazario = pd.read_csv(f'{email_path}/Nazario.csv', encoding='latin-1')
    df_nifr = pd.read_csv(f'{email_path}/Nigerian_Fraud.csv', encoding='latin-1')
    df_spas = pd.read_csv(f'{email_path}/SpamAssasin.csv', encoding='latin-1')
except FileNotFoundError as e:
    print(f"Error loading file for Merge 2: {e}")

#1. Merge 1: Combine Enron and Ling (mdf_1)
mdf_1 = pd.concat([df_enron, df_ling], ignore_index=True)
print(f"Merged mdf_1 size: {len(mdf_1)}")

# 2. Merge 2: Combine CEAS, Nazario, NifR, and SpAs (mdf_2)
mdf_2 = pd.concat([df_ceas, df_nazario, df_nifr, df_spas], ignore_index=True)
print(f"Merged mdf_2 size: {len(mdf_2)}")

# 3. Final Integration: Combine mdf_1 and mdf_2 into the final dataset
# Note: This will create many NaN (missing) values for columns mdf_1 doesn't have (like sender, date).

# I solved that later in the code 
df_full = pd.concat([mdf_1, mdf_2], ignore_index=True)

print(f"\n--- Final Full Dataset Created ---")
print(f"Total records in df_full: {len(df_full)}")
print("Columns in the final dataset (Note NaNs will be present):")
print(df_full.columns.tolist())

# Now df_full is ready for the feature engineering steps (Date, URL, Sender Entropy).

Merged mdf_1 size: 32626
Merged mdf_2 size: 49860

--- Final Full Dataset Created ---
Total records in df_full: 82486
Columns in the final dataset (Note NaNs will be present):
['subject', 'body', 'label', 'sender', 'receiver', 'date', 'urls']


In [ ]:
# Group 1: Datasets that have only 3 colunms (Subject, Body, Label)
try:
    df_enron = pd.read_csv('/kaggle/input/phishing-email-dataset/Enron.csv', encoding='latin-1', usecols=['subject', 'body', 'label'])
    df_ling = pd.read_csv('/kaggle/input/phishing-email-dataset/Ling.csv', encoding='latin-1', usecols=['subject', 'body', 'label'])
except FileNotFoundError as e:
    print(f"Error loading file for Merge 1: {e}")

# Group 2: Rich Metadata Datasets that have 7 colunms (Sender, Date, Subject, Body, Label, etc.)
try:
    df_ceas = pd.read_csv('/kaggle/input/phishing-email-dataset/CEAS_08.csv', encoding='latin-1')
    df_nazario = pd.read_csv('/kaggle/input/phishing-email-dataset/Nazario.csv', encoding='latin-1')
    df_nifr = pd.read_csv('/kaggle/input/phishing-email-dataset/Nigerian_Fraud.csv', encoding='latin-1')
    df_spas = pd.read_csv('/kaggle/input/phishing-email-dataset/SpamAssasin.csv', encoding='latin-1')
except FileNotFoundError as e:
    print(f"Error loading file for Merge 2: {e}")

# 1. Merge 1: Combine Enron and Ling (mdf_1)
mdf_1 = pd.concat([df_enron, df_ling], ignore_index=True)
print(f"Merged mdf_1 size: {len(mdf_1)}")

# 2. Merge 2: Combine CEAS, Nazario, NifR, and SpAs (mdf_2)
mdf_2 = pd.concat([df_ceas, df_nazario, df_nifr, df_spas], ignore_index=True)
print(f"Merged mdf_2 size: {len(mdf_2)}")

# 3. Final Integration: Combine mdf_1 and mdf_2 into the final dataset
# Note: This will create many NaN (missing) values for columns mdf_1 doesn't have (like sender, date).

# I solved that later in the code 
df_full = pd.concat([mdf_1, mdf_2], ignore_index=True)

print(f"\n--- Final Full Dataset Created ---")
print(f"Total records in df_full: {len(df_full)}")
print("Columns in the final dataset (Note NaNs will be present):")
print(df_full.columns.tolist())

# Now df_full is ready for the feature engineering steps (Date, URL, Sender Entropy).

In [4]:
# Show full text in cells
pd.set_option('display.max_colwidth', None)

# Update FILE_PATH to use the dynamic email_path from kagglehub
FILE_PATH = f'{email_path}/phishing_email.csv'

if not os.path.exists(FILE_PATH):
    print(f"Error: File not found at '{FILE_PATH}'.")
else:
    try:
        # Loading with latin-1 encoding as specified
        df = pd.read_csv(FILE_PATH, encoding='latin-1')

        print("File loaded successfully.")
        print(f"Total emails: {len(df)}")

        # Get 5 full-text samples 
        # Note: In 'phishing_email.csv', the column is 'text_combined'
        sample_df = df[['text_combined', 'label']].sample(n=5, random_state=42)

        display(sample_df)  # Show complete email text

        print("\nLabel Key: 1 = Phishing, 0 = Legitimate")

    except Exception as e:
        print(f"An error occurred: {e}")

File loaded successfully.
Total emails: 82486


,text_combined,label
30381,endangered languages workshop foundation endangered languages pleased announce first workshop entitled steps language rescue take place university york weekend 26 27 july year programme saturday 2 00 2 30 arrival late registration 2 30 2 40 introduction foundation endangered languages fel committee session 2 40 3 10 endangered language policy india mahendra verma 3 10 3 40 situation berber languages north africa farid aitsiselmi 3 40 4 10 script groups use particular areas john clews session ii 4 30 5 00 izhorian estonia sweden language revival possible ilya nikolaev 5 00 5 30 issues standardisation tsimshian language american north west tonya nicole stebbins 5 30 6 00 overview endangered languages brunei darussalam peter martin break dinner followed agm general meeting foundation sunday session iii 9 00 9 30 language revival case irish belfast alison henry 9 30 10 00 gaelic endangered language problems prospects kenneth mackinnon session iv 10 30 11 00 planning kurdish language linguistics siamak rezaei durroei 11 00 11 30 big oil threat minority languages andes nicholas ostler 11 30 12 00 final discussion round policy pointers noon lunch departures non members foundation welcome attend though course possible join site registration york workshop steps language rescue 2627 july 1997 conference take place university york accommodation arranged derwent college near campus university york located south medieval walled city york comfortable walking distance railway alternatively bus number 4 5 caught station getting derwent college stop accomodation 30 pounds without 20 pounds ensuite shower w c meals saturday night 7 pounds sunday lunch 6 pounds let us know dietary requirements might registration fees unwaged e g students unemployed 10 pounds standard 20 pounds please provide evidence confirmation apply lower rate total _ _ _ pounds please send form dr mahendra verma membership secretary fel language linguistic science univ york york yo1 5dd england also possible register person door however plan would help inform us advance dr verma email address mkv1 york ac uk mahendra verma nicholas ostler managing director president linguacubun ltd foundation endangered languages http www bris ac uk depts philosophy ctll fel batheaston villa 172 bailbrook lane bath ba1 7aa england 441225 852865 fax 441225 859258 nostler chibcha demon co uk,0
64974,claretta claretta_bordersfusemailcom cialis wont break wallet senior fellow faculty member james macgregor burns academy leadership university maryland board directors maryland association counties eastern regional representative national association black county officials former director education close foundation former president maryland municipal league karthikeyan popular tamil name also name god karthikeyan murugan second son lord httpkralpealcom however hours later said ive got much cultivating football squad launch 1967 rosko initially recorded shows france midday spin programme first midday spin show rosko introduced like emperor geeter heater leader groovy host west coast clear skin mess mind itll make feel good highlighted new motown reggae rock music thu 07 aug 2008 222319 0000 looking remedy best source,1
62961,roger upole schkeramsncom kyle rickey wrote im trying get file version executable file network times network quite slow prevent blocking main application decided put code separate thread depending network speed take 30 seconds return vpn connection etc put separate thread block entire application function returns take following code example import threading time os win32api file rnetwork_sharefileexe def get_version_number ospathexistsfile info win32apigetfileversioninfofile ms infofileversionms ls infofileversionls return win32apihiwordms win32apilowordms win32apihiwordls win32apilowordls def test print joinstri get_version_number print aa threadingthreadtargetteststart 1 print aa timesleep05 output looks like aa aa 20015 aa problem 1st aas got printe


Label Key: 1 = Phishing, 0 = Legitimate


In [5]:
mdf_1.head(2)

,subject,body,label
0,"hpl nom for may 25 , 2001",( see attached file : hplno 525 . xls )\r\n- hplno 525 . xls,0
1,re : nom / actual vols for 24 th,"- - - - - - - - - - - - - - - - - - - - - - forwarded by sabrae zajac / hou / ect on 05 / 30 / 2001 12 : 07 pm - - - - - - - - - - - - - - - - - - - - - - - - - - -\r\nenron capital & trade resources corp . from : "" eileen ponton "" 05 / 29 / 2001 08 : 37 am\r\nto : davilal @ txu . com , cstonel @ txu . com , mjones 7 @ txu . com , hpl . scheduling @ enron . com , liz . bellamy @ enron . com , szajac @ enron . com\r\ncc :\r\nsubject : re : nom / actual vols for 24 th\r\ni will agree with your nomination of 33 . 750\r\n- - - - - forwarded by eileen ponton / houston / pefs / pec on 05 / 29 / 01 08 : 36 am\r\n- - - - -\r\n"" charlie\r\nstone "" to : "" eileen ponton ""\r\n, "" melissa jones ""\r\ncom > , hpl . scheduling @ enron . com , liz . bellamy @ enron . com ,\r\nszajac @ enron . com\r\n05 / 25 / 01 subject : re : nom / actual vols for 24 th\r\n04 : 23 pm\r\nwe do not agree with the "" nominated "" volume .\r\nour records reflect the following nom . schedule .\r\n30 rate , eff . 0900 hrs through hour beginning 1400 hrs .\r\n6 . 250\r\n60 rate , eff . 1400 hrs through hour beginning 1700 hrs .\r\n7 . 500\r\n30 rate , eff . 1700 hrs through hour beginning 0900 hrs .\r\n20 . 000\r\ntotal nominated 33 . 750\r\nplease review your source data and let us know if you agree with the above .\r\nthanks ,\r\nccs\r\n"" eileen ponton "" on 05 / 25 / 2001 04 : 06 : 50 pm\r\nto : david avila / lsp / enserch / us @ tu , charlie stone / energy / txu @ tu , melissa\r\njones / energy / txu @ tu , hpl . scheduling @ enron . com , liz . bellamy @ enron . com ,\r\nszajac @ enron . com\r\ncc :\r\nsubject : nom / actual vols for 24 th\r\nnom mcf mmbtu\r\n27 , 500 33 , 109 34 , 003",0


In [6]:
mdf_2.head(3)

,sender,receiver,date,subject,body,label,urls
0,Young Esposito <Young@iworld.de>,user4@gvc.ceas-challenge.cc,"Tue, 05 Aug 2008 16:31:02 -0700",Never agree to be a loser,"Buck up, your troubles caused by small dimension will soon be over!\nBecome a lover no woman will be able to resist!\nhttp://whitedone.com/\n\n\ncome. Even as Nazi tanks were rolling down the streets, the dreamersphilosopher or a journalist. He was still not sure.I do the same.",1,1
1,Mok <ipline's1983@icable.ph>,user2.2@gvc.ceas-challenge.cc,"Tue, 05 Aug 2008 18:31:03 -0500",Befriend Jenna Jameson,\nUpgrade your sex and pleasures with these techniques http://www.brightmade.com\n\n\n,1,1
2,Daily Top 10 <Karmandeep-opengevl@universalnet.psi.br>,user2.9@gvc.ceas-challenge.cc,"Tue, 05 Aug 2008 20:28:00 -1200",CNN.com Daily Top 10,">+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+= >THE DAILY TOP 10 >from CNN.com >Top videos and stories as of: Aug 1, 2008 3:58 PM EDT >+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+= TOP 10 VIDEOS 1. MONTAUK 'MONSTER' http://www.cnn.com/video/partners/email/index.html?url=/video/us/2008/07/31/moos.montauk.monster.cnn Is it a devil dog? Is it a turtle? Is it the Montauk Monster? CNN's Jeanne Moos asks, ""what is this thing?"" 2. RACY PHOTOS OF TODDLER'S MOM http://www.cnn.com/video/partners/email/index.html?url=/video/crime/2008/07/31/ng.racy.photos.cnn 3. NEWS OF THE ABSURD EPISODE 54 http://www.cnn.com/video/partners/email/index.html?url=/video/podcasts/absurd/site/2008/08/01/nota.episode.54.cnn 4. POLICE BEATING DISPUTE http://www.cnn.com/video/partners/email/index.html?url=/video/us/2008/07/31/levs.police.video.cnn 5. MOM PLEADS FOR GIRL'S RETURN http://www.cnn.com/video/partners/email/index.html?url=/video/crime/2008/08/01/hill.boss.reigh.plea.cnn 6. DEFENDANT FAKES HEART ATTACK http://www.cnn.com/video/partners/email/index.html?url=/video/crime/2008/08/01/dnt.fake.heart.attack.mxf.whio 7. KILLER CARRIED VICTIM'S HEAD http://www.cnn.com/video/partners/email/index.html?url=/video/world/2008/07/31/natpkg.can.bus.decapitation.ctv 8. MURDER CONFESSION RECANTED http://www.cnn.com/video/partners/email/index.html?url=/video/world/2008/08/01/blake.brazil.teen.murder.itn 9. ANTHRAX SUSPECT'S HOME http://www.cnn.com/video/partners/email/index.html?url=/video/us/2008/08/01/von.dr.bruce.ivins.home.cnn 10. HECKLERS INTERRUPT OBAMA TALK http://www.cnn.com/video/partners/email/index.html?url=/video/politics/2008/08/01/sot.fl.obama.protesters.baynews9 TOP 10 STORIES 1. SUSPECT IN BEHEADING IDENTIFIED http://www.cnn.com/2008/WORLD/americas/08/01/canada.beheading/index.html Canadian police say Vince Weiguang Li, 40, of Edmonton is charged with second-degree murder in the beheading of a man on a bus. 2. JUDGE TAKEN OFF LAST JENA 6 CASES http://www.cnn.com/2008/CRIME/08/01/jena6.appeal/index.html 3. PEOPLE MAG GETS PITT-JOLIE PIX http://www.cnn.com/2008/SHOWBIZ/Movies/08/01/brangelina.photos.ap/index.html 4. ATTACK IN TORONTO CALLED RACIAL http://www.cnn.com/2008/WORLD/americas/08/01/canada.attack.ap/index.html 5. IREPORTERS' UNUSUAL NAMES http://www.cnn.com/2008/LIVING/08/01/unusual.names.irpt/index.html 6. MOTHER PLEADS FOR CHILD'S RETURN http://www.cnn.com/2008/CRIME/08/01/rockefeller.kidnapping/index.html 7. KARADZIC: I MADE DEAL WITH U.S. http://www.cnn.com/2008/WORLD/europe/08/01/karadzic.trial/index.html 8. SUSPECT ARRESTED IN SWIM KILLINGS http://www.cnn.com/2008/CRIME/08/01/wisconsin.shooting.ap/index.html 9. ANTHRAX SUSPECT APPARENT SUICIDE http://www.cnn.com/2008/CRIME/08/01/anthrax.death/index.html 10. MCCAIN: OBAMA CRITICISM 'FAIR' http://www.cnn.com/2008/POLITICS/08/01/campaign.wrap/index.html CNN, The Most Trusted Name in News > Cable News Network LP, LLLP. < > One CNN Center, Atlanta, Georgia 30303 < > 2008 Cable News Network LP, LLLP. < > A Time Warner Company. < > All Rights Reserved. < ========================================================= = Please send comments or suggestions by going to = = http://www

In [7]:
mdf_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32626 entries, 0 to 32625
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   subject  32366 non-null  object
 1   body     32626 non-null  object
 2   label    32626 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 764.8+ KB


In [8]:
mdf_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49860 entries, 0 to 49859
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sender    49529 non-null  object
 1   receiver  47768 non-null  object
 2   date      49377 non-null  object
 3   subject   49773 non-null  object
 4   body      49859 non-null  object
 5   label     49860 non-null  int64 
 6   urls      49860 non-null  int64 
dtypes: int64(2), object(5)
memory usage: 2.7+ MB


In [9]:
#count mdf_1 & 2 null values
mdf_1.isnull().sum()

subject    260
body         0
label        0
dtype: int64

In [10]:
mdf_2.isnull().sum()

sender       331
receiver    2092
date         483
subject       87
body           1
label          0
urls           0
dtype: int64

In [11]:
print("mdf_1 columns:", mdf_1.columns.tolist())
print("mdf_2 columns:", mdf_2.columns.tolist())
print(f"Columns in df_full: {df_full.columns.tolist()}")

mdf_1 columns: ['subject', 'body', 'label']
mdf_2 columns: ['sender', 'receiver', 'date', 'subject', 'body', 'label', 'urls']
Columns in df_full: ['subject', 'body', 'label', 'sender', 'receiver', 'date', 'urls']


In [12]:
df_full

subject  \
0                               hpl nom for may 25 , 2001   
1                        re : nom / actual vols for 24 th   
2              enron actuals for march 30 - april 1 , 201   
3                               hpl nom for may 30 , 2001   
4                               hpl nom for june 1 , 2001   
...                                                   ...   
82481                       Busy? Home Study Makes Sense!   
82482              Preferred Non-Smoker Rates for Smokers   
82483  How to get 10,000 FREE hits per day to any website   
82484                                 Cannabis Difference   
82485                               [ILUG] WILSON  KAMELA   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

In [13]:
# Handle NaNs in the source columns:
df_full['subject'] = df_full['subject'].fillna('')
df_full['body'] = df_full['body'].fillna('')
df_full['sender'] = df_full['sender'].fillna('')
df_full['date'] = df_full['date'].fillna('')


# Create the final combined text input for the Transformer model (RoBERTa)
df_full['combined_text_for_bert'] = (
    df_full['sender'] + " [SEP] " +
    df_full['date'] + " [SEP] " +
    df_full['subject'] + " [SEP] " +
    df_full['body']
)

In [14]:
df_full

subject  \
0                               hpl nom for may 25 , 2001   
1                        re : nom / actual vols for 24 th   
2              enron actuals for march 30 - april 1 , 201   
3                               hpl nom for may 30 , 2001   
4                               hpl nom for june 1 , 2001   
...                                                   ...   
82481                       Busy? Home Study Makes Sense!   
82482              Preferred Non-Smoker Rates for Smokers   
82483  How to get 10,000 FREE hits per day to any website   
82484                                 Cannabis Difference   
82485                               [ILUG] WILSON  KAMELA   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

In [15]:
!pip install tabulate
# Define the IP Regex just for checking the extracted list elements
ip_regex = r'^\s*http[s]?://\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}'

# Define a general URL regex to get all URLs in a list and get also their count 
url_regex = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

# Extract all URLs in each email into a column called 'urls_in_email' 
df_full['urls_in_email'] = df_full['body'].apply(lambda x: re.findall(url_regex, str(x)))

#URL Cont: take the length of urls in urls_in_email column
df_full['url_count_derived'] = df_full['urls_in_email'].apply(len).fillna(0).astype(int)

def check_for_ip(url_list):
    """Returns 1 if any URL in the list starts with an IP address, 0 otherwise."""
    if not url_list:
        return 0
        
    for url in url_list:
        if re.match(ip_regex, url):
            return 1
    return 0

df_full['has_ip_url_derived'] = df_full['urls_in_email'].apply(check_for_ip)

print(" Final URL Numerical Features Derived.")

print(df_full[['label', 'url_count_derived', 'has_ip_url_derived', 'urls_in_email']].head(10).to_markdown(index=False))

 Final URL Numerical Features Derived.
|   label |   url_count_derived |   has_ip_url_derived | urls_in_email   |
|--------:|--------------------:|---------------------:|:----------------|
|       0 |                   0 |                    0 | []              |
|       0 |                   0 |                    0 | []              |
|       0 |                   0 |                    0 | []              |
|       0 |                   0 |                    0 | []              |
|       0 |                   0 |                    0 | []              |
|       0 |                   0 |                    0 | []              |
|       0 |                   0 |                    0 | []              |
|       0 |                   0 |                    0 | []              |
|       0 |                   0 |                    0 | []              |
|       0 |                   0 |                    0 | []              |


In [16]:
# Access the list of URLs for the email at index 82481 why that row ?
#I just noticed it has alot of urls
urls_for_email_82481 = df_full.iloc[82481]['urls_in_email']

print(f"URLs found in email 82481: {urls_for_email_82481}")

URLs found in email 82481: ['http://images.pcdi-homestudy.com/ads/e277/unisex_header.gif)', 'http://images.pcdi-homestudy.com/ads/e277/1pixel.gif)', 'http://images.pcdi-homestudy.com/ads/e277/1pixel.gif)', 'http://images.pcdi-homestudy.com/ads/e277/1pixel.gif)', 'http://images.pcdi-homestudy.com/ads/e277/1pixel.gif)', 'http://images.pcdi-homestudy.com/ads/e277/1pixel.gif)', 'http://images.pcdi-homestudy.com/ads/e277/unisex_5up.jpg)', 'http://images.pcdi-homestudy.com/ads/e277/1pixel.gif)', 'http://images.pcdi-homestudy.com/ads/e277/1pixel.gif)', 'http://images.pcdi-homestudy.com/ads/e277/1pixel.gif)', 'http://www.frugaljoe.com/logo.jpg)', 'http://www.frugaljoe.com/dot.gif)', 'http://www.frugaljoe.com/unsubscribe.php?eid=396068\\', 'http://www.330w.com/open/open.php?eid=jm@netnoteinc.com&oid=Professional_Career_Development_Institute-1-12032002-HTML&custid=frugaljoe)']


In [17]:
df_full

subject  \
0                               hpl nom for may 25 , 2001   
1                        re : nom / actual vols for 24 th   
2              enron actuals for march 30 - april 1 , 201   
3                               hpl nom for may 30 , 2001   
4                               hpl nom for june 1 , 2001   
...                                                   ...   
82481                       Busy? Home Study Makes Sense!   
82482              Preferred Non-Smoker Rates for Smokers   
82483  How to get 10,000 FREE hits per day to any website   
82484                                 Cannabis Difference   
82485                               [ILUG] WILSON  KAMELA   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

In [18]:
#Checking:
condition_urls_missing = df_full['urls'].isnull()

condition_urls_found = df_full['urls_in_email'].str.len() > 0

df_found_in_body_only = df_full[condition_urls_missing & condition_urls_found]

print(f"Total rows found where 'urls' was missing but a URL was found in the body: {len(df_found_in_body_only)}")

Total rows found where 'urls' was missing but a URL was found in the body: 0


In [19]:
df_full.iloc[82481,:]

subject                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

In [20]:
# check if there is row has ip_url_derived with value>0
df_full[df_full['has_ip_url_derived']>0]

subject  \
32777                                                    Your order   
33507                                 [soaplite] Digest Number 1746   
39621                                Virtualization is here and now   
39650  [UAI] Special Session - Advances in Processing Temporal Data   
40537      Re: [python-win32] Windows Vista slow down Python xmlrpc   
...                                                             ...   
82348                                  Would you like a $250 check?   
82400                   >> Best rates on mortgage in the country! <   
82409                                 [ILUG] Re: whats up -colonize   
82455                                    The best possible mortgage   
82478                               Reverse Aging While Burning Fat   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [21]:
def domain_entropy(domain):
    """Calculates Shannon Entropy of a domain string."""
    domain_str = str(domain)
    if not domain_str or domain_str == 'nan':
        return 0.0
    domain_part = domain_str.split('@')[-1]
    probabilities = [domain_part.count(c) / len(domain_part) for c in set(domain_part)]
    entropy = -sum(p * log2(p) for p in probabilities if p > 0)
    return entropy

#Calculate Heuristic Features 

# Sender Entropy
df_full['sender_entropy'] = df_full['sender'].apply(domain_entropy)

# Time-Based Features
df_full['date_parsed'] = pd.to_datetime(df_full['date'], errors='coerce', utc=True)
df_full['is_weekend'] = df_full['date_parsed'].dt.dayofweek.apply(
    lambda x: 1 if x >= 5 and not pd.isna(x) else 0)
df_full['outside_business_hrs'] = df_full['date_parsed'].dt.hour.apply(
    lambda x: 1 if not pd.isna(x) and (x < 9 or x > 17) else 0)

# URL Features 
url_regex = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
ip_regex = r'http[s]?://\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}'

# We calculate the derived features from the body directly
df_full['url_count_derived'] = df_full['body'].str.count(url_regex).fillna(0)
df_full['has_ip_url_derived'] = df_full['body'].str.contains(ip_regex).fillna(False).astype(int)


#Final Cleanup & Verification 

text_col = 'combined_text_for_bert'
numerical_cols = [
    'sender_entropy',
    'is_weekend',
    'outside_business_hrs',
    'url_count_derived',
    'has_ip_url_derived'
]
target_col = 'label'

# Ensure all inputs are 100% clean
df_full[numerical_cols] = df_full[numerical_cols].fillna(0)
df_full = df_full.dropna(subset=[target_col])

print("Feature Engineering Complete.")
print(f"Columns ready for model: {numerical_cols} + ['{text_col}']")
print(f"Total samples: {len(df_full)}")

Feature Engineering Complete.
Columns ready for model: ['sender_entropy', 'is_weekend', 'outside_business_hrs', 'url_count_derived', 'has_ip_url_derived'] + ['combined_text_for_bert']
Total samples: 82486


In [22]:
#Define Function to Extract Local Hour 
def get_local_hour(date_str):
    """
    Extracts the hour (0-23) directly from the string to preserve local time.
    Finds patterns like '13:19:58' -> returns 13.
    """
    # Regex to find HH:MM:SS pattern
    match = re.search(r'\s(\d{1,2}):\d{2}:\d{2}', str(date_str))
    if match:
        return int(match.group(1))
    return np.nan  # Return NaN if no time found

# Extract the LOCAL hour 
df_full['local_hour'] = df_full['date'].apply(get_local_hour)

# Fill missing hours with a safe default (e.g., 12 noon) or 0
# Filling with 9 (start of business day) is a safe neutral choice to avoid flagging as suspicious
df_full['local_hour'] = df_full['local_hour'].fillna(9).astype(int)

# Recalculate 'outside_business_hrs' ---

# Now check if THAT local hour is outside 9-17 (9 AM - 5 PM)
df_full['outside_business_hrs'] = df_full['local_hour'].apply(
    lambda x: 1 if (x < 9 or x > 17) else 0
)

# Verification 
print("Local Hour Logic Applied.")

print(df_full[['date', 'local_hour', 'outside_business_hrs']].head())

Local Hour Logic Applied.
  date  local_hour  outside_business_hrs
0                9                     0
1                9                     0
2                9                     0
3                9                     0
4                9                     0


In [23]:
df_full

subject  \
0                               hpl nom for may 25 , 2001   
1                        re : nom / actual vols for 24 th   
2              enron actuals for march 30 - april 1 , 201   
3                               hpl nom for may 30 , 2001   
4                               hpl nom for june 1 , 2001   
...                                                   ...   
82481                       Busy? Home Study Makes Sense!   
82482              Preferred Non-Smoker Rates for Smokers   
82483  How to get 10,000 FREE hits per day to any website   
82484                                 Cannabis Difference   
82485                               [ILUG] WILSON  KAMELA   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

In [24]:
# Define Inputs
text_data = df_full['combined_text_for_bert'].values
numerical_data = df_full[numerical_cols].values
labels = df_full['label'].values

# X_temp contains the remaining 85% (Train + Validation)
X_temp_text, X_test_text, X_temp_num, X_test_num, y_temp, y_test = train_test_split(
    text_data, numerical_data, labels,
    test_size=0.15,
    random_state=42,
    stratify=labels
)

# Separate TRAIN and VALIDATION
# 0.15 / 0.85 = ~0.176
X_train_text, X_val_text, X_train_num, X_val_num, y_train, y_val = train_test_split(
    X_temp_text, X_temp_num, y_temp,
    test_size=0.176,
    random_state=42,
    stratify=y_temp
)

# Scale Numerical Features
#Fit on Train. Transform Val and Test using Train's stats (mean and standard deviation)
scaler = StandardScaler()
train_num_scaled = scaler.fit_transform(X_train_num)
val_num_scaled = scaler.transform(X_val_num)
test_num_scaled = scaler.transform(X_test_num)

print("Data Split into 3 Sets:")
print(f"  Training:   {len(y_train)} samples (For learning)")
print(f"  Validation: {len(y_val)} samples (For Early Stopping)")
print(f"  Testing:    {len(y_test)} samples (For final evaluation)")


Data Split into 3 Sets:
  Training:   57773 samples (For learning)
  Validation: 12340 samples (For Early Stopping)
  Testing:    12373 samples (For final evaluation)


In [25]:
# Initialize Tokenizer
#'bert-base-uncased' as it's the standard for English text classification
MODEL_NAME = 'roberta-base'
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

#Define Custom Hybrid Dataset Class
class HybridPhishingDataset(Dataset):
    def __init__(self, texts, numerical_feats, labels, tokenizer, max_len=128):
        self.texts = texts
        self.numerical_feats = numerical_feats
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        numeric = self.numerical_feats[idx]
        label = self.labels[idx]

        # Tokenize the text
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,    # Add '[CLS]' and '[SEP]'
            max_length=self.max_len,
            padding='max_length',       # Pad sentences to same length
            truncation=True,            # Truncate too long sentences
            return_attention_mask=True,
            return_tensors='pt'         # Return PyTorch tensors
        )

        # Return a dictionary containing all inputs
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'numerical_features': torch.tensor(numeric, dtype=torch.float),
            'label': torch.tensor(label, dtype=torch.long)
        }

# Create DataLoaders

# Parameters
BATCH_SIZE = 16
MAX_LEN = 128

# Create 3 Datasets 
train_dataset = HybridPhishingDataset(X_train_text, train_num_scaled, y_train, tokenizer, MAX_LEN)
val_dataset   = HybridPhishingDataset(X_val_text, val_num_scaled, y_val, tokenizer, MAX_LEN)  # NEW
test_dataset  = HybridPhishingDataset(X_test_text, test_num_scaled, y_test, tokenizer, MAX_LEN)

# Create 3 Loaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
val_loader   = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)   # NEW
test_loader  = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

print("3 DataLoaders Ready: train_loader, val_loader, test_loader")

3 DataLoaders Ready: train_loader, val_loader, test_loader


In [26]:
class HybridRoBERTaClassifier(nn.Module):
    def __init__(self, numerical_input_dim, hidden_dim=768, num_classes=2, dropout_prob=0.3):
        super(HybridRoBERTaClassifier, self).__init__()

        # Load Pre-trained RoBERTa
        # This handles the text understanding
        self.roberta = RobertaModel.from_pretrained('roberta-base')
        self.dropout = nn.Dropout(dropout_prob)

        # Define the Final Classifier Layer
        # Input Size = RoBERTa Output (768) + Numerical Features (5)
        self.classifier = nn.Linear(hidden_dim + numerical_input_dim, num_classes)

    def forward(self, input_ids, attention_mask, numerical_features):
        # Pass text through RoBERTa
        outputs = self.roberta(input_ids=input_ids, attention_mask=attention_mask)

        # Extract the 'pooler_output' (The representation of the [CLS] token)
        # This vector summarizes the entire email text
        text_embedding = outputs.pooler_output  # Shape: [batch_size, 768]
        text_embedding = self.dropout(text_embedding)

        # Concatenate Text Embedding with Numerical Features
        # join them side-by-side, Shape becomes: [batch_size, 768 + 5]
        
        combined_features = torch.cat((text_embedding, numerical_features), dim=1)

        # Pass through final classifier to get predictions
        logits = self.classifier(combined_features)

        return logits

print("Hybrid RoBERTa Model Architecture defined.")

Hybrid RoBERTa Model Architecture defined.


In [27]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Initialize the Model
NUMERICAL_FEATURES_COUNT = 5
model = HybridRoBERTaClassifier(numerical_input_dim=NUMERICAL_FEATURES_COUNT)
model.to(device) # Move model to GPU

# Setup Optimizer
optimizer = AdamW(model.parameters(), lr=2e-5, weight_decay=1e-2)

# Setup Loss Function
loss_fn = nn.CrossEntropyLoss()

# Setup Scheduler
EPOCHS = 5
total_steps = len(train_loader) * EPOCHS

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

print("Model, Optimizer (PyTorch AdamW), and GPU setup complete.")

Using device: cuda


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/ha33ona/jupyterlab/.venv/lib/python3.12/site-packages/torch/cuda/__init__.py:283: UserWarning: 
    Found GPU0 NVIDIA GB10 which is of cuda capability 12.1.
    Minimum and Maximum cuda capability supported by this version of PyTorch is
    (8.0) - (12.0)
    
  warnings.warn(


Model, Optimizer (PyTorch AdamW), and GPU setup complete.


In [ ]:
def format_time(elapsed):
    elapsed_rounded = int(round(elapsed))
    return str(datetime.timedelta(seconds=elapsed_rounded))

#TRAINING FUNCTION 
def train_epoch_clean(model, data_loader, optimizer, device, scheduler, loss_fn):
    model.train()
    total_train_loss = 0
    predictions = []
    true_labels = []

    t0 = time.time()

    for step, batch in enumerate(data_loader):
        # Unpack batch
        b_input_ids = batch['input_ids'].to(device)
        b_input_mask = batch['attention_mask'].to(device)
        b_numerical = batch['numerical_features'].to(device)
        b_labels = batch['label'].to(device)

        # Forward & Backward
        model.zero_grad()
        logits = model(b_input_ids, b_input_mask, b_numerical)
        loss = loss_fn(logits, b_labels)
        total_train_loss += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()

        # Store for metrics
        preds = torch.argmax(logits, dim=1).flatten()
        predictions.extend(preds.cpu().numpy())
        true_labels.extend(b_labels.cpu().numpy())

    # Calculate metrics
    avg_train_loss = total_train_loss / len(data_loader)
    train_acc = accuracy_score(true_labels, predictions)
    epoch_duration = format_time(time.time() - t0)

    return avg_train_loss, train_acc, epoch_duration

#VALIDATION FUNCTION 
def eval_model_clean(model, data_loader, device, loss_fn):
    model.eval()
    total_val_loss = 0
    predictions = []
    true_labels = []
    t0 = time.time()

    with torch.no_grad():
        for batch in data_loader:
            b_input_ids = batch['input_ids'].to(device)
            b_input_mask = batch['attention_mask'].to(device)
            b_numerical = batch['numerical_features'].to(device)
            b_labels = batch['label'].to(device)

            logits = model(b_input_ids, b_input_mask, b_numerical)
            loss = loss_fn(logits, b_labels)
            total_val_loss += loss.item()

            preds = torch.argmax(logits, dim=1).flatten()
            predictions.extend(preds.cpu().numpy())
            true_labels.extend(b_labels.cpu().numpy())

    avg_val_loss = total_val_loss / len(data_loader)
    val_acc = accuracy_score(true_labels, predictions)
    val_f1 = f1_score(true_labels, predictions)
    val_time = format_time(time.time() - t0)

    return avg_val_loss, val_acc, val_f1, val_time

print(f"🚀 Starting training for {EPOCHS} epochs...")
print("-" * 80)
print(f"{'Epoch':^5} | {'Status':^10} | {'Time':^8} | {'Loss':^6} | {'Acc':^6} | {'F1':^6}")
print("-" * 80)

best_val_loss = float('inf')
patience_counter = 0
EARLY_STOPPING_PATIENCE = 3

for epoch_i in range(0, EPOCHS):

    #  TRAIN (Uses train_loader)
    train_loss, train_acc, train_time = train_epoch_clean(model, train_loader, optimizer, device, scheduler, loss_fn)
    print(f"{epoch_i + 1:^5} | {'Train':^10} | {train_time:^8} | {train_loss:.4f} | {train_acc:.4f} | {'-':^6}")

    # VALIDATE (Uses val_loader)
    val_loss, val_acc, val_f1, val_time = eval_model_clean(model, val_loader, device, loss_fn)
    print(f"{'':^5} | {'Valid':^10} | {val_time:^8} | {val_loss:.4f} | {val_acc:.4f} | {val_f1:.4f}")

    #  EARLY STOPPING CHECK (Checks val_loss)
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
        torch.save(model.state_dict(), 'best_hybrid_roberta_model.pth')
        print(f"{'':^5} | {'SAVE':^10} | Best Model Saved (Loss improved)")
    else:
        patience_counter += 1
        print(f"{'':^5} | {'WAIT':^10} | Patience {patience_counter}/{EARLY_STOPPING_PATIENCE}")

    print("-" * 80)

    if patience_counter >= EARLY_STOPPING_PATIENCE:
        print("\nEarly Stopping Triggered.")
        break

🚀 Starting training for 5 epochs...
--------------------------------------------------------------------------------
Epoch |   Status   |   Time   |  Loss  |  Acc   |   F1  
--------------------------------------------------------------------------------


In [ ]:
def evaluate_performance(model, data_loader, device):
    model.eval()
    predictions = []
    true_labels = []

    # Get Predictions from the Model
    print("Running predictions on Test Set...")
    with torch.no_grad():
        for batch in data_loader:
            b_input_ids = batch['input_ids'].to(device)
            b_input_mask = batch['attention_mask'].to(device)
            b_numerical = batch['numerical_features'].to(device)
            b_labels = batch['label'].to(device)

            logits = model(b_input_ids, b_input_mask, b_numerical)

            # Convert logits to actual class predictions (0 or 1)
            preds = torch.argmax(logits, dim=1).flatten()

            predictions.extend(preds.cpu().numpy())
            true_labels.extend(b_labels.cpu().numpy())

    # Calculate Metrics
    acc = accuracy_score(true_labels, predictions)
    f1 = f1_score(true_labels, predictions)

    print("\n" + "="*30)
    print(f"FINAL TEST RESULTS")
    print("="*30)
    print(f"Accuracy: {acc:.4f}")
    print(f"F1-Score: {f1:.4f}")

    # Detailed Classification Report
    print("\n--- Classification Report ---")
    print(classification_report(true_labels, predictions, target_names=['Legitimate', 'Phishing']))

    # Plot Confusion Matrix
    cm = confusion_matrix(true_labels, predictions)

    plt.figure(figsize=(6, 5))
    # 'd' format is for integers (counts), 'Blues' is a clean color map
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False,
                xticklabels=['Legitimate', 'Phishing'],
                yticklabels=['Legitimate', 'Phishing'])
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.title('Confusion Matrix')
    plt.show()

checkpoint_path = 'best_hybrid_roberta_model.pth'

try:
    model.load_state_dict(torch.load(checkpoint_path))
    print(f"Loaded best model weights from: {checkpoint_path}")
except FileNotFoundError:
    print("Model file not found. Testing with current model weights instead.")

evaluate_performance(model, test_loader, device)

In [ ]:
pth_filename = 'best_hybrid_roberta_model.pth' 
pkl_filename = 'classifier.pkl'                


print(f"Loading best weights from {pth_filename}...")
if os.path.exists(pth_filename):
    model.load_state_dict(torch.load(pth_filename))
    print("Best model weights loaded successfully.")
else:
    print(f"Warning: {pth_filename} not found. Using current model state.")
    torch.save(model.state_dict(), pth_filename)


print("\n Running Final Evaluation on Test Set...")

evaluate_performance(model, test_loader, device)

print(f"\n Saving full model to {pkl_filename}...")

torch.save(model, pkl_filename)
print("Full model object saved.")

# GENERATE DOWNLOAD LINKS
print("\n⬇️ Download Links (Click to download):")
display(FileLink(pth_filename))
display(FileLink(pkl_filename))
